In [1]:
import pandas as pd
import ydata_profiling as yp
# from dateutil import parser as ps

df = pd.read_excel('original/ventas_ultimos_6_meses.xlsx')
# df.info()

c:\Users\admin\Documents\acoding\HanovaCaso2\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dfi = df.map(str)  # convierte el reporte inicial todo a texto para poder mostrarlo sin errores
dfi.to_excel('sample/ventas_ultimos_6_meses_object.xlsx', index=False, engine='openpyxl')

In [3]:
# Quantity_Sold

# Normalizar 
def normalize_quantity(q):
    q = str(q).strip().lower()

    if q in ['dos', '2']:
        return 2.0
    if q in ['uno', 'una', '1']:
        return 1.0
    if 'unidades' in q: # '20 unidades'
        try:
            return float(q.split()[0]) # '20'
        except (ValueError, IndexError):
            return None
    try:
        return float(q)
    except ValueError:
        return None

df['Quantity_Sold'] = df['Quantity_Sold'].apply(normalize_quantity)

In [4]:
# Sale_Date
df['Sale_Date'] = pd.to_datetime(df['Sale_Date'], errors='coerce', dayfirst=True)

# Aqui como tenemos fechas invalidas, si usamos la solución de productos_departamento 
# como sea intenta usar el parser de dateutil al no poder convertir a datetime las fechas invalidas

# print(df)
print("Num fechas NaT:", df['Sale_Date'].isna().sum())


Num fechas NaT: 14


C:\Users\admin\AppData\Local\Temp\ipykernel_13676\247807814.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Sale_Date'] = pd.to_datetime(df['Sale_Date'], errors='coerce', dayfirst=True)


In [5]:
# Payment_Method
df['Payment_Method'] = df['Payment_Method'].str.lower().replace({
    'tarjita' : 'card',
    'tarjta' : 'card',
    'tarjeta' : 'card',
    'efectivo' : 'cash'
}).str.title()

In [6]:
# Creamos columna 'Returned' para indicar si la venta fue devuelta (-1 = devolución)
returned = df['Quantity_Sold'] == -1
df.insert(3, 'Returned', returned)

In [7]:
# Generar reporte pandas profiling
report = yp.ProfileReport(
            df,
            title="Reporte Ventas Ultimos 6 Meses",
            explorative=True,
            minimal=True,
        )
report.to_file("limpio/reporte_ventas_ultimos_6_meses.html")

Export report to file: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [8]:
# num_duplicates = df.duplicated().sum()
# print(f"Num filas duplicadas: {num_duplicates}")
# df = df.drop_duplicates()

df.to_excel('limpio/ventas_ultimos_6_meses_clean.xlsx', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order_ID        400 non-null    object        
 1   SKU             400 non-null    object        
 2   Quantity_Sold   400 non-null    float64       
 3   Returned        400 non-null    bool          
 4   Sale_Date       386 non-null    datetime64[ns]
 5   Total_Amount    400 non-null    float64       
 6   Payment_Method  400 non-null    object        
dtypes: bool(1), datetime64[ns](1), float64(2), object(3)
memory usage: 19.3+ KB
